In [0]:
from google.colab import drive
import os
CODE_PATH='/content/drive/My Drive/TCC/Codes/Codigos Atualizados/'
DATA_PATH = '/content/drive/My Drive/TCC/EMG_Database/'
drive.mount('/content/drive', force_remount=True)
if not os.getcwd() == CODE_PATH:
  os.chdir(CODE_PATH)
# google.colab.drive module has a recently-added flush_and_unmount()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


algorithms.py	     dataset.py		    LICENSE
cleaners.py	     denoising_example.py   ModelCollection.py
create_dataframe.py  emg_loader.py	    plot3d.py
Dataset		     Experiments	    preprocessing.py
dataset_gen.py	     feature_extraction.py  README.md


In [51]:
!git push

Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [0]:
import os
import pandas as pd
import numpy as np
from os.path import join
import joblib

experiments_path = "/content/drive/My Drive/TCC/Experiments/Final"

folders = "NEURAL_INTER", "NEURAL_INTRA", "LOGISTIC_REGRESSION_INTER", "LOGISTIC_REGRESSION_INTRA", "KNN_INTER", "KNN_INTRA", "NEURAL_BACKWARD", "LOGISTIC_BACKWARD", "NEURAL_INTER_2", "LOGISTIC_REGRESSION_INTER_2", "NEURAL_INTER_500", "LOGISTIC_REGRESSION_INTER_500", "KNN_INTER_500"

results = []

# Balanced Accuracy: 0.5761904761904761 1.0 0.9713778646426674 0.06975002100540188
# F1 Score: 0.652892561983471 1.0 0.9720400063040288 0.06013857133284821
# Cohen Kappa: 0.17204301075268824 1.0 0.9391710437706268 0.13901191091982332
# AUC: 0.5761904761904763 1.0 0.9713778646426674 0.06975002100540188
# Precision: 0.6515151515151515 1.0 0.9747144277553079 0.058112464179509916
# Recall: 0.6818181818181818 1.0 0.9726883113352995 0.056428652557790546

metrics = ["Balanced Accuracy", "F1 Score", "Cohen Kappa", "AUC", "Precision", "Recall"]
types = ["Min", "Max", "Mean", "Std"]


def read_results(filepath):
  with open(filepath, "r") as file:
    data = file.readlines()
  results = []
  results.append(int(data[0].split(": ")[1]))
  for line in data[1:]:
    broke = line.split(":")
    values = broke[1].split(" ")
    for k in values:
      if len(k)>0: results.append(float(k))
  return results

def add_info(log, info):
  log["Experiment"] = info[0]
  log["N"] = info[1]
  log["Fold"] = info[2]
  return log

def load_classes(path):
  with open(os.path.join(path, "classes.txt")) as file:
    classes = file.readlines()
  classes = np.array(classes).astype(int)
  encoder = joblib.load(os.path.join(path, "encoder.joblib"))
  classes = encoder.inverse_transform(classes)
  return classes
  
logs = []

folds = ("train", "test", "valid")

for folder in folders:
  print(f"LOADING {folder} ", end="")
  classes_exps = os.listdir(os.path.join(experiments_path, folder))
  for classes_exp in classes_exps:
    exp_path = os.path.join(experiments_path, folder, classes_exp)
    n = int(classes_exp.split("_")[-1])
    print(n, end=", ")
    for fold in folds:
      try:
        classes = load_classes(exp_path)
        line = [folder, n, fold] + read_results(os.path.join(exp_path, f"{fold}_results.txt")) + [classes]
        try:
          log = pd.read_csv(os.path.join(exp_path, "log.csv"))
          log = add_info(log, [folder, n, fold])
          logs.append(log)
        except:
          pass
        results.append(line)
      except:
        pass
  print("... Done!")
logs = pd.concat(logs, axis=0)
columns_names = ["Experiment", "N", "Fold", "Examples"] + [metric + " " + type_ for metric in metrics for type_ in types] + ["Classes"]
results = pd.DataFrame(data=results, columns=columns_names)

LOADING NEURAL_INTER 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING NEURAL_INTRA 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING LOGISTIC_REGRESSION_INTER 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING LOGISTIC_REGRESSION_INTRA 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING KNN_INTER 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING KNN_INTRA 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING NEURAL_BACKWARD 49, 45, 41, 37, 33, 29, 25, 21, 17, 13, 9, 5, ... Done!
LOADING LOGISTIC_BACKWARD 49, 45, 41, 37, 33, 29, 25, 21, 17, 13, 9, 5, ... Done!
LOADING NEURAL_INTER_2 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING LOGISTIC_REGRESSION_INTER_2 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING NEURAL_INTER_500 2, 3, 4, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, ... Done!
LOADING LOGISTIC_REGRES

In [0]:
results.Experiment = results.Experiment.apply(lambda x: x.replace("LOGISTIC_REGRESSION", "LOGISTIC"))

In [0]:
def remove_last(txt):
  return " ".join(txt.split(" ")[:-1])

cols = [6,10,14,18,22,26]
def to_txt(df, experiment, fold="test"):
  s = []
  df = df[(df.Experiment==experiment) & (df.Fold==fold)]
  for c in cols:
    s.append(df.iloc[:,[c, c+1]].T.apply(lambda x: str(round(100*x[0],1)) +" ± "+ str(round(100*x[1],1))))
  s = np.vstack(s)
  s = pd.DataFrame(s.T, columns = [remove_last(x) for x in df.columns[cols]])
  s = pd.concat([df.iloc[:, [0, 1, 2]].reset_index().drop("index", axis=1), s], axis=1)
  return s

tables = []

for exp in results.Experiment.unique():
  exp_table = to_txt(results, exp, fold="test")
  tables.append(exp_table)

tables[-1].head(10)

,Experiment,N,Fold,Balanced Accuracy,F1 Score,Cohen Kappa,AUC,Precision,Recall
0,KNN_INTER_500,2,test,98.0 ± 2.5,98.0 ± 2.5,95.9 ± 5.1,98.0 ± 2.5,98.1 ± 2.4,98.0 ± 2.5
1,KNN_INTER_500,3,test,92.8 ± 7.4,93.1 ± 7.1,89.3 ± 10.9,94.3 ± 5.7,94.6 ± 5.1,93.1 ± 7.1
2,KNN_INTER_500,4,test,79.6 ± 6.3,75.2 ± 9.4,68.2 ± 10.4,83.6 ± 5.3,82.9 ± 5.3,76.4 ± 8.0
3,KNN_INTER_500,5,test,67.9 ± 7.9,63.1 ± 10.1,55.0 ± 11.0,77.1 ± 5.5,69.3 ± 7.7,64.2 ± 9.3
4,KNN_INTER_500,9,test,53.7 ± 7.4,50.0 ± 9.6,46.1 ± 9.0,73.1 ± 4.6,59.9 ± 8.7,52.4 ± 8.0
5,KNN_INTER_500,13,test,43.0 ± 6.0,39.7 ± 7.3,37.6 ± 7.1,68.8 ± 3.6,47.0 ± 4.6,42.6 ± 6.6
6,KNN_INTER_500,17,test,39.4 ± 5.8,36.0 ± 5.9,34.1 ± 5.8,67.0 ± 2.9,43.0 ± 5.9,38.1 ± 5.4
7,KNN_INTER_500,21,test,33.1 ± 4.8,29.2 ± 4.2,27.6 ± 4.5,63.8 ± 2.3,36.0 ± 3.5,31.3 ± 4.2
8,KNN_INTER_500,25,test,27.9 ± 4.7,24.1 ± 4.1,23.0 ± 4.1,61.5 ± 2.1,30.8 ± 5.0,26.2 ± 3.9
9,KNN_INTER_500,29,test,25.3 ± 5.4,21.3 ± 3.8,20.5 ± 4.2,60.3 ± 2.1,27.3 ± 5.7,23.4 ± 4.0


In [0]:
!pip install pylatex
import pylatex
from pylatex import Document, Subsection, Tabular, Table, NoEscape
import sys

tables_path = "/content/drive/My Drive/TCC/Tables/Experiments Results"

try: os.mkdir(tables_path)
except: pass

cols = [1, 3, 4, 5, 6, 7, 8]
for tab in tables:
  exp_name = tab.Experiment.unique()[0]
  table = Table()
  table.append(NoEscape("[H]"))
  table.append(NoEscape("\centering"))
  table.add_caption("Resultados de " + exp_name)
  table.append(NoEscape("\label{tab:" + exp_name.lower() + "_results}"))
  table.append(NoEscape("\\resizebox{\\textwidth}{!}{"))
#   with table.create(Tabular("|c|c|c|c|c|c|c|", pos="!h")) as tabular:
  with table.create(Tabular("|c|c|c|c|c|c|c|")) as tabular:
    tabular.add_hline()
    tabular.add_row(tab.columns[cols])
    for row in tab.values:
      tabular.add_hline()
      tabular.add_row(row[cols])
    tabular.add_hline()
  table.append(NoEscape("}"))
  table.generate_tex(os.path.join(tables_path, exp_name.lower() + "_results"))

In [0]:
bar_model_folder = '/content/drive/My Drive/TCC/Images/Bar_Model_Comparisons'
line_model_folder = '/content/drive/My Drive/TCC/Images/Line_Model_Comparisons'
bar_fold_folder = '/content/drive/My Drive/TCC/Images/Bar_Fold_Comparisons'
line_fold_folder = '/content/drive/My Drive/TCC/Images/Line_Fold_Comparisons'
logs_folder = '/content/drive/My Drive/TCC/Images/Logs_Train'

for folder in (bar_model_folder, line_model_folder, bar_fold_folder, line_fold_folder, logs_folder):
  try:
    os.mkdir(folder)
    print(os.listdir(folder))
  except:
    pass
# mkdir '/content/drive/My Drive/TCC/Images/Logs_Train'

[]
[]
[]
[]
[]


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

plt.style.use("seaborn-dark")

def fold2name(fold):
  d = {"train":"treino", "test":"teste", "valid":"validação"}
  return d[fold]

def plot_3_comparisons_bar(kind, fold="test", metric="Balanced Accuracy", use_knn=True, save_folder=None):
  ctt_nn = (results.Experiment==f"NEURAL_{kind}") & (results.Fold==fold)
  ctt_knn = (results.Experiment==f"KNN_{kind}") & (results.Fold==fold)
  ctt_lr = (results.Experiment==f"LOGISTIC_{kind}") & (results.Fold==fold)
  names = "Neural network", "K-Nearest Neighbors", "Logistic Regression"    
  
  labels = results[ctt_nn]["N"]
  nn = results[ctt_nn]
  knn = results[ctt_knn]
  lr= results[ctt_lr]

  def expand(x):
    n = len(labels)
    app = [0]*(n-len(x))
    return np.array(list(x) + app)

  nn_errs = nn[f"{metric} Std"]
  lr_errs = lr[f"{metric} Std"]
  knn_errs = expand(knn[f"{metric} Std"])
  nn_metrics = nn[f"{metric} Mean"]
  lr_metrics = lr[f"{metric} Mean"]
  knn_metrics = expand(knn[f"{metric} Mean"])
  width = 0.3  # the width of the bars
  
  x = np.arange(len(labels)//2 + 1)

  if use_knn:
    w1, w2, w3 = x - width, x, x + width
  else:
    w1, w2 = x - width/2, x + width/2

  fig, axes = plt.subplots(2, 1, figsize=(18,8))
#   rects1 = ax[0].bar(w1, nn_metrics, width, yerr=nn_errs, label='Neural Network')
#   rects2 = ax[0].bar(w2, lr_metrics, width, yerr=lr_errs, label='Logistic Regression')
#   if use_knn: rects3 = ax[0].bar(w3, knn_metrics, width, yerr=knn_errs, label='k-NN')

  tam = len(nn_metrics)
  l = len(nn_metrics)//2
  
  rects11 = axes[0].bar(w1[:l], nn_metrics[:l], width, yerr=nn_errs[:l], label='Neural Network')
  rects12 = axes[0].bar(w2[:l], lr_metrics[:l], width, yerr=lr_errs[:l], label='Logistic Regression')
  if use_knn: 
    rects13 = axes[0].bar(w3[:l], knn_metrics[:l], width, yerr=knn_errs[:l], label='k-NN')
    
  rects21 = axes[1].bar(w1[:tam-l], nn_metrics[l:], width, yerr=nn_errs[l:], label='Neural Network')
  rects22 = axes[1].bar(w2[:tam-l], lr_metrics[l:], width, yerr=lr_errs[l:], label='Logistic Regression')
  if use_knn:
    rects23 = axes[1].bar(w3[:tam-l], knn_metrics[l:], width, yerr=knn_errs[l:], label='k-NN')
    

  # Add some text for labels, title and custom x-axis tick labels, etc.
  if use_knn:
    h00, h01 = knn_metrics[:l].min()-knn_errs[:l].max()-.1,nn_metrics[:l].max()+nn_errs[:l].max()
    h10, h11 = knn_metrics[l:].min()-knn_errs[l:].max()-.1,nn_metrics[l:].max()+nn_errs[l:].max()
  else:
    h00, h01 = nn_metrics[:l].min()-nn_errs[:l].max()-.1, nn_metrics[:l].max()+nn_errs[:l].max()
    h10, h11 = nn_metrics[l:].min()-nn_metrics[l:].max()-.1, nn_metrics[l:].max()+nn_errs[l:].max()

  h00, h10 = max(0, h00), max(0, h10)
#   h01, h11 = min(1, h10), min(1, h11)
  fontsize=20
  
  for ax, labels_ in zip(axes, (labels[:l], labels[l:])):
    ax.set_ylabel('Porcentagem', fontsize=fontsize)
    if ax is axes[0]: ax.set_title(f'{kind}: {metric} para o conjunto de {fold2name(fold)}', fontsize=fontsize)
    ax.set_xticklabels([0]+list(labels_), fontsize=fontsize)
    ax.set_yticks(np.arange(0,1.1,.1))
    ax.set_yticklabels(np.arange(0,101,10), fontsize=fontsize)
    if ax is axes[1]: ax.set_xlabel("Nº de classes", fontsize=fontsize)
#     ax.legend(bbox_to_anchor=(1,1), fontsize="xx-large")
    ax.legend(loc="best", fontsize=fontsize)
    if ax is axes[0]: ax.set_ylim([h00, h01])
    else: ax.set_ylim([h10, h11])
    ax.grid()
    
  def autolabel(ax, rects, errs, scores, h=0):
      """Attach a text label above each bar in *rects*, displaying its height."""
      span = ax.get_ybound()
      dif = span[1]-span[0]
      for rect, err, score in zip(rects, errs, scores):
          height = rect.get_height()
          ax.annotate('{}'.format(round(100*score,1)),
                      xy=(rect.get_x() + rect.get_width() / 2, span[0]+.04*dif/.5),
                      xytext=(0, 3),
                      textcoords="offset points",
                      ha='center', va='bottom', fontweight="bold", size=16, alpha=0.7)
          ax.annotate('±{}'.format(round(100*err,1)),
                      xy=(rect.get_x() + rect.get_width() / 2, span[0]+.01*dif/.5),
                      xytext=(0, 3),
                      textcoords="offset points",
                      ha='center', va='bottom', fontweight="bold", size=16, alpha=0.7)

  
#   autolabel(axes[0], rects11, nn_errs[:l], nn_metrics[:l], h00)
#   autolabel(axes[0], rects12, lr_errs[:l], lr_metrics[:l], h00)
#   if use_knn: autolabel(axes[0], rects13, knn_errs[:l], knn_metrics[:l], h00)
#   autolabel(axes[1], rects21, nn_errs[l:], nn_metrics[l:], h10)
#   autolabel(axes[1], rects22, lr_errs[l:], lr_metrics[l:], h10)
#   if use_knn: autolabel(axes[1], rects23, knn_errs[l:], knn_metrics[l:], h10)

  fig.tight_layout()

  plt.show()
  
  if save_folder != None:
    metric_ = metric.replace(" ", "_")
    name = f"{kind}_{metric_}_{fold2name(fold)}".lower()
    save_name = os.path.join(save_folder, name + "_model_bar_comparison")
    fig.savefig(save_name+".png")
    with open(save_name + ".tex", "w") as file:
      text = """\\begin{figure}[H]
    \centering
    \includegraphics[width=0.97\\textwidth]{Images/%s.png}
    \caption{Comparação da métrica \\textit{%s} no conjunto de %s para %s}
    \label{fig:%s}
\end{figure}""" % (save_name.split("/")[-1], metric.lower(), fold2name(fold), kind.replace("_", "\_"), save_name.split("/")[-1])
      file.write(text)
      
for metric in metrics:
  plot_3_comparisons_bar("INTER_500", metric=metric, save_folder=bar_model_folder)
  plot_3_comparisons_bar("INTER", metric=metric, save_folder=bar_model_folder)
  plot_3_comparisons_bar("INTRA", metric=metric, save_folder=bar_model_folder)
  plot_3_comparisons_bar("BACKWARD", metric=metric, use_knn=False, save_folder=bar_model_folder)
  plot_3_comparisons_bar("INTER_2", metric=metric, use_knn=False, save_folder=bar_model_folder)


Output hidden; open in https://colab.research.google.com to view.

In [0]:
def plot_3_folds_bar(kind, experiment, metric="Balanced Accuracy", reverse=False, save_folder=None):
  ctt_train = (results.Experiment==f"{experiment}_{kind}") & (results.Fold=="train")
  ctt_valid = (results.Experiment==f"{experiment}_{kind}") & (results.Fold=="valid")
  ctt_test = (results.Experiment==f"{experiment}_{kind}") & (results.Fold=="test")
  names = "Train", "Validation", "Test"

  labels = results[ctt_train]["N"]
  train = results[ctt_train]
  valid = results[ctt_valid]
  test = results[ctt_test]
    
  train_errs = train[f"{metric} Std"]
  valid_errs = valid[f"{metric} Std"]
  test_errs = test[f"{metric} Std"]
  train_metrics = train[f"{metric} Mean"]
  valid_metrics = valid[f"{metric} Mean"]
  test_metrics = test[f"{metric} Mean"]

  width = .3  # the width of the bars  
  

  x = np.arange(len(labels)//2 + 1)

  w1, w2, w3 = x - width, x, x + width

  fig, axes = plt.subplots(2, 1, figsize=(18,8))
#   rects1 = ax[0].bar(w1, nn_metrics, width, yerr=nn_errs, label='Neural Network')
#   rects2 = ax[0].bar(w2, lr_metrics, width, yerr=lr_errs, label='Logistic Regression')
#   if use_knn: rects3 = ax[0].bar(w3, knn_metrics, width, yerr=knn_errs, label='k-NN')

  tam = len(train_metrics)
  l = len(train_metrics)//2
  
  rects11 = axes[0].bar(w1[:l], train_metrics[:l], width, yerr=train_errs[:l], label='Treino')
  rects12 = axes[0].bar(w2[:l], valid_metrics[:l], width, yerr=valid_errs[:l], label='Validação')
  rects13 = axes[0].bar(w3[:l], test_metrics[:l], width, yerr=test_errs[:l], label='Teste')
    
  rects21 = axes[1].bar(w1[:tam-l], test_metrics[l:], width, yerr=train_errs[l:], label='Treino')
  rects22 = axes[1].bar(w2[:tam-l], valid_metrics[l:], width, yerr=valid_errs[l:], label='Validação')
  rects23 = axes[1].bar(w3[:tam-l], test_metrics[l:], width, yerr=test_errs[l:], label='Teste')
    

  # Add some text for labels, title and custom x-axis tick labels, etc.
  h00, h01 = test_metrics[:l].min()-test_errs[:l].max()-.1, train_metrics[:l].max()+train_errs[:l].max()
  h10, h11 = test_metrics[l:].min()-test_errs[l:].max()-.1, train_metrics[l:].max()+train_errs[l:].max()
  print(h00, h01, h10, h11)

  h00, h10 = max(0, h00), max(0, h10)
#   h01, h11 = min(1, h10), min(1, h11)
  
  fontsize=20
  
  for ax, labels_ in zip(axes, (labels[:l], labels[l:])):
    ax.set_ylabel('Porcentagem', fontsize=fontsize)
#     if ax is axes[0]: ax.set_title(f'{kind}: {metric} para o conjunto de {fold2name(fold)}')
    if ax is axes[0]: ax.set_title(f"{experiment}_{kind}: {metric}", fontsize=fontsize)
    ax.set_xticklabels([0]+list(labels_), fontsize=fontsize)
    ax.set_yticks(np.arange(0,1.1,.1))
    ax.set_yticklabels(np.arange(0,101,10), fontsize=fontsize)
#     ax.legend(bbox_to_anchor=(1,1), fontsize="xx-large")
    ax.legend(loc="best", fontsize=fontsize)
    ax.grid()
  
  axes[1].set_xlabel("Nº de classes", fontsize=fontsize)
  axes[0].set_ylim([h00, h01])
  axes[1].set_ylim([h10, h11])
    
  def autolabel(ax, rects, errs, scores, h=0):
      """Attach a text label above each bar in *rects*, displaying its height."""
      span = ax.get_ybound()
      dif = span[1]-span[0]
      for rect, err, score in zip(rects, errs, scores):
          height = rect.get_height()
          ax.annotate('{}'.format(round(100*score,1)),
                      xy=(rect.get_x() + rect.get_width() / 2, span[0]+.04*dif/.5),
                      xytext=(0, 3),
                      textcoords="offset points",
                      ha='center', va='bottom', fontweight="bold", size=16, alpha=0.7)
          ax.annotate('±{}'.format(round(100*err,1)),
                      xy=(rect.get_x() + rect.get_width() / 2, span[0]+.01*dif/.5),
                      xytext=(0, 3),
                      textcoords="offset points",
                      ha='center', va='bottom', fontweight="bold", size=16, alpha=0.7)

  
#   autolabel(axes[0], rects11, train_errs[:l], train_metrics[:l], h00)
#   autolabel(axes[0], rects12, valid_errs[:l], valid_metrics[:l], h00)
#   autolabel(axes[0], rects13, test_errs[:l], test_metrics[:l], h00)
#   autolabel(axes[1], rects21, train_errs[l:], train_metrics[l:], h10)
#   autolabel(axes[1], rects22, valid_errs[l:], valid_metrics[l:], h10)
#   autolabel(axes[1], rects23, test_errs[l:], test_metrics[l:], h10)
  
  plt.tight_layout()
  
  
  
  
  
  
  
  
  
  plt.show()
  
  if save_folder != None:
    metric_ = metric.replace(" ", "_")
    name = f"{experiment}_{kind}"
    save_name = os.path.join(save_folder, f"{experiment}_{kind}_{metric_}".lower() + "_fold_bar_comparison")
    print_name = save_name.split("/")[-1]
    fig.savefig(save_name+".png")
    with open(save_name + ".tex", "w") as file:
      text = """\\begin{figure}[H]
    \centering
    \includegraphics[width=0.97\\textwidth]{Images/%s.png}
    \caption{Comparação da métrica \\textit{%s} entre os folds em %s}
    \label{fig:%s}
\end{figure}""" % (print_name, metric.lower(), name.replace("_", "\_"), print_name)
      file.write(text)
      
for metric in metrics:
  plot_3_folds_bar("INTER","LOGISTIC", metric=metric, save_folder=bar_fold_folder)
  plot_3_folds_bar("INTER_2","LOGISTIC", metric=metric, save_folder=bar_fold_folder)
  plot_3_folds_bar("INTRA","LOGISTIC", metric=metric, save_folder=bar_fold_folder)
  
  plot_3_folds_bar("INTER","NEURAL", metric=metric, save_folder=bar_fold_folder)
  plot_3_folds_bar("INTER_2","NEURAL", metric=metric, save_folder=bar_fold_folder)
  plot_3_folds_bar("INTRA","NEURAL", metric=metric, save_folder=bar_fold_folder)
  
  plot_3_folds_bar("BACKWARD","LOGISTIC", metric=metric, reverse=False, save_folder=bar_fold_folder)
  plot_3_folds_bar("BACKWARD","NEURAL", metric=metric, reverse=False, save_folder=bar_fold_folder)
  
  plot_3_folds_bar("INTER_500","LOGISTIC", metric=metric, save_folder=bar_fold_folder)
  plot_3_folds_bar("INTER_500","NEURAL", metric=metric, save_folder=bar_fold_folder)
#   plot_3_folds_bar("INTER_500","KNN", metric=metric, save_folder=bar_fold_folder)
  

Output hidden; open in https://colab.research.google.com to view.

In [0]:
results[results.Experiment=="KNN_INTER_500"].N


462     2
463     3
464     4
465     5
466     9
467    13
468    17
469    21
470    25
471    29
472    33
473    37
474    41
475    45
476    49
Name: N, dtype: int64

In [0]:
def plot_3_folds_line(kind, experiment, metric="Balanced Accuracy", reverse=False, save_folder=None):
  ctt_train = (results.Experiment==f"{experiment}_{kind}") & (results.Fold=="train")
  ctt_valid = (results.Experiment==f"{experiment}_{kind}") & (results.Fold=="valid")
  ctt_test = (results.Experiment==f"{experiment}_{kind}") & (results.Fold=="test")
  names = "Train", "Validation", "Test"

  labels = results[ctt_train]["N"]
  train = results[ctt_train]
  valid = results[ctt_valid]
  test = results[ctt_test]
  print(train.shape, valid.shape, test.shape)

  train_errs = train[f"{metric} Std"]
  valid_errs = valid[f"{metric} Std"]
  test_errs = test[f"{metric} Std"]
  train_metrics = train[f"{metric} Mean"]
  valid_metrics = valid[f"{metric} Mean"]
  test_metrics = test[f"{metric} Mean"]
  print(train_metrics.shape, valid_metrics.shape, test_metrics.shape)
  print(train_errs.shape, valid_errs.shape, test_errs.shape)

  width = 0.3  # the width of the bars
  
  x = np.arange(len(labels))
  fontsize=20
  
  fig, ax = plt.subplots(figsize=(12,6))
  delta = .1
  line1 = ax.plot(x-delta, train_metrics, label='Treino', color="g")
  line2 = ax.plot(x, valid_metrics, label='Validação', color="b")
  line3 = ax.plot(x+delta, test_metrics, label='Teste', color="r")
  ax.legend(fontsize=fontsize)
  p1 = ax.scatter(x-delta, train_metrics, color="g", s=20)
  p2 = ax.scatter(x, valid_metrics, color="b", s=20)
  p3 = ax.scatter(x+delta, test_metrics, color="r", s=20)
  
  ax.set_xlabel("Nº de classes", fontsize=fontsize)
  
  ax.set_ylabel('Porcentagem', fontsize=fontsize)
  ax.set_title(f"{experiment}_{kind}: {metric}", fontsize=fontsize)
  ax.set_xticks(x)
  ax.set_xticklabels(labels, fontsize=fontsize)
  
  def autolabel(lines, i, errs, scores):
      """Attach a text label above each bar in *rects*, displaying its height."""
      color = lines[0].get_color()
      print(lines)
      xd = i*0.0
      for x, y, err, score in zip(lines[0].get_xdata(), lines[0].get_ydata(), errs, scores):
          ax.plot([x+xd, x+xd], [y-err/2, y+err/2], color=color, linewidth=3, alpha=.7)
          ax.plot([x-0.1+xd, xd+x+0.1], [y-err/2]*2, color=color, linewidth=3, alpha=.7)
          ax.plot([x-0.1+xd, xd+x+0.1], [y+err/2]*2, color=color, linewidth=3, alpha=.7)
  
  autolabel(line1, -1, train_errs, train_metrics)
  autolabel(line2, 0, valid_errs, valid_metrics)
  autolabel(line3, 1, test_errs, test_metrics)
  fig.tight_layout()

  plt.grid()
  plt.show()


  if save_folder != None:
    metric_ = metric.replace(" ", "_")
    name = f"{experiment}_{kind}"
    save_name = os.path.join(save_folder, f"{experiment}_{kind}_{metric_}".lower() + "_fold_line_comparison")
    print_name = save_name.split("/")[-1]
    fig.savefig(save_name+".png")
    with open(save_name + ".tex", "w") as file:
      text = """\\begin{figure}[H]
    \centering
    \includegraphics[width=0.97\\textwidth]{Images/%s.png}
    \caption{Comparação da métrica %s entre os folds em %s}
    \label{fig:%s}
\end{figure}""" % (print_name, metric, name.replace("_", "\_"), print_name)
      file.write(text)
      
logs_ = logs[logs.Fold=="train"].drop("Fold", axis=1)

def plot_log(kind, experiment, n_classes=2, save_folder=None):
  ctt = (logs_.Experiment==f"{experiment}_{kind}") & (logs_.N == n_classes)
  if ctt.sum()==0: return
  data = logs_[ctt]
    
  label = data.epoch
  train_loss = data.loss
  valid_loss = data.val_loss
  train_acc = data.acc
  valid_acc = data.val_acc

  fontsize=10
  
  fig, ax = plt.subplots(figsize=(10, 6))
  plt.grid()
  line1 = ax.plot(label, train_loss, label='Erro no treino', color="b")
  line2 = ax.plot(label, valid_loss, label='Erro na validação', color="r")
  ax2 = ax.twinx()
  ax2.plot(label, train_acc, "--", label='Acurácia no treino', color="k")
  ax2.plot(label, valid_acc, "--", label='Acurácia na validação', color="y")

  ax.legend(loc=2, fontsize=fontsize)
  ax2.legend(loc=3, fontsize=fontsize)
#   ax.legend(bbox_to_anchor=(1,1))
#   ax2.legend(bbox_to_anchor=(1,.9))
  
  ax2.set_yticklabels((ax2.get_yticks() * 100).astype(int), fontsize=fontsize)
  ax.set_ylabel('Erro', fontsize=fontsize)
  ax2.set_ylabel('Acurácia [%]', fontsize=fontsize)
  ax.set_title(f"{experiment}_{kind} - {n_classes} classes: Erro e acurácia durante o treino", fontsize=fontsize)
  ax.set_xlabel("Época", fontsize=fontsize)
#   ax.set_xticks(label)
#   ax.set_xticklabels(labels)

  fig.tight_layout()

  plt.show()
  
  if save_folder != None:
    name = f"{experiment}_{kind}"
    save_name = os.path.join(save_folder, f"{experiment}_{kind}_{n_classes}".lower() + "_logs")
    fig.savefig(save_name+".png")
    with open(save_name + ".tex", "w") as file:
      text = """\\begin{figure}[H]
    \centering
    \includegraphics[width=0.47\\textwidth]{Images/%s.png}
    \caption{Erro e acurácia durante o treinamento: %s, %s classes}
    \label{fig:%s}
\end{figure}""" % (save_name.split("/")[-1], name.replace("_", "\_"), n_classes, save_name.split("/")[-1])
      file.write(text)

for metric in metrics:
  plot_3_folds_line("INTER","LOGISTIC", metric=metric, save_folder=line_fold_folder)
  plot_3_folds_line("INTER_2","LOGISTIC", metric=metric, save_folder=line_fold_folder)
  plot_3_folds_line("INTRA","LOGISTIC", metric=metric, save_folder=line_fold_folder)
  
  plot_3_folds_line("INTER","NEURAL", metric=metric, save_folder=line_fold_folder)
  plot_3_folds_line("INTER_2","NEURAL", metric=metric, save_folder=line_fold_folder)
  plot_3_folds_line("INTRA","NEURAL", metric=metric, save_folder=line_fold_folder)
  
  plot_3_folds_line("BACKWARD","LOGISTIC", metric=metric, reverse=False, save_folder=line_fold_folder)
  plot_3_folds_line("BACKWARD","NEURAL", metric=metric, reverse=False, save_folder=line_fold_folder)
  
  plot_3_folds_line("INTER_500","LOGISTIC", metric=metric, save_folder=line_fold_folder)
  plot_3_folds_line("INTER_500","NEURAL", metric=metric, save_folder=line_fold_folder)
#   plot_3_folds_line("INTER_500","KNN", metric=metric, save_folder=line_fold_folder)
  

Output hidden; open in https://colab.research.google.com to view.

In [0]:
for i in range(2, 50):
  plot_log("INTER","NEURAL", n_classes=i, save_folder=logs_folder)
  plot_log("INTER_500","NEURAL", n_classes=i, save_folder=logs_folder)
  plot_log("INTER_2","NEURAL", n_classes=i, save_folder=logs_folder)
  plot_log("INTRA","NEURAL", n_classes=i, save_folder=logs_folder)
  
  plot_log("INTER","LOGISTIC", n_classes=i, save_folder=logs_folder)
  plot_log("INTER_500","LOGISTIC", n_classes=i, save_folder=logs_folder)
  plot_log("INTER_2","LOGISTIC", n_classes=i, save_folder=logs_folder)
  plot_log("INTRA","LOGISTIC", n_classes=i, save_folder=logs_folder)
  
  plot_log("BACKWARD","NEURAL", n_classes=i, save_folder=logs_folder)
  plot_log("BACKWARD","LOGISTIC", n_classes=i, save_folder=logs_folder)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_3_comparisons_line(kind, fold="test", metric="Balanced Accuracy", use_knn=True, save_folder=None):
  ctt_nn = (results.Experiment==f"NEURAL_{kind}") & (results.Fold==fold)
  ctt_knn = (results.Experiment==f"KNN_{kind}") & (results.Fold==fold)
  ctt_lr = (results.Experiment==f"LOGISTIC_{kind}") & (results.Fold==fold)
  names = "Neural network", "K-Nearest Neighbors", "Logistic Regression"    
  
  labels = results[ctt_nn]["N"]
  nn = results[ctt_nn]
  knn = results[ctt_knn]
  lr= results[ctt_lr]

  def expand(x):
    n = len(labels)
    app = [0]*(n-len(x))
    return np.array(list(x) + app)

  nn_errs = nn[f"{metric} Std"]
  lr_errs = lr[f"{metric} Std"]
  knn_errs = expand(knn[f"{metric} Std"])
  nn_metrics = nn[f"{metric} Mean"]
  lr_metrics = lr[f"{metric} Mean"]
  knn_metrics = expand(knn[f"{metric} Mean"])
  width = 0.3  # the width of the bars
  
  x = np.arange(len(labels))
    
                
  # Add some text for labels, title and custom x-axis tick labels, etc.
#   if use_knn:
#     h00, h01 = knn_metrics[:l].min()-knn_errs[:l].max()-.1,nn_metrics[:l].max()+nn_errs[:l].max()
#     h10, h11 = knn_metrics[l:].min()-knn_errs[l:].max()-.1,nn_metrics[l:].max()+nn_errs[l:].max()
#   else:
#     h00, h01 = nn_metrics[:l].min()-nn_errs[:l].max()-.1, nn_metrics[:l].max()+nn_errs[:l].max()
#     h10, h11 = nn_metrics[l:].min()-nn_metrics[l:].max()-.1, nn_metrics[l:].max()+nn_errs[l:].max()


  fontsize=20
  
  
  fig, ax = plt.subplots(figsize=(14, 6))
  delta = .1
  line1 = ax.plot(x-delta, nn_metrics, label='Neural Network', color="g", alpha=0.7)
  line2 = ax.plot(x, lr_metrics, label='Logistic Regression', color="b", alpha=0.7)
  if use_knn: line3 = ax.plot(x+delta, knn_metrics, label='k-NN', color="r", alpha=0.7)
  ax.legend(fontsize=fontsize)
  p1 = ax.scatter(x-delta, nn_metrics, color="g", s=25, alpha=.3)
  p2 = ax.scatter(x, lr_metrics, color="b", s=25, alpha=.3)
  if use_knn: p3 = ax.scatter(x+delta, knn_metrics, color="r", s=25, alpha=.3)

  ax.set_xlabel("Nº de classes", fontsize=fontsize)
  
  ax.set_ylabel('Porcentagem', fontsize=fontsize)
  ax.set_yticks(np.arange(0,1.01,.1))
  ax.set_yticklabels(np.arange(0,101,10), fontsize=fontsize)
  ax.set_title(f'{kind}: {metric} para o conjunto de {fold2name(fold)}', fontsize=fontsize)
  ax.set_xticks(x)
  ax.set_xticklabels(labels, fontsize=fontsize)
  def autolabel(lines, i, errs, scores):
      """Attach a text label above each bar in *rects*, displaying its height."""
      color = lines[0].get_color()
      print(lines)
      xd = i*0.0
      for x, y, err, score in zip(lines[0].get_xdata(), lines[0].get_ydata(), errs, scores):
          ax.plot([x+xd, x+xd], [y-err/2, y+err/2], color=color, linewidth=3, alpha=.7)
          ax.plot([x-0.1+xd, xd+x+0.1], [y-err/2]*2, color=color, linewidth=3, alpha=.7)
          ax.plot([x-0.1+xd, xd+x+0.1], [y+err/2]*2, color=color, linewidth=3, alpha=.7)
  
  autolabel(line1, -1, nn_errs, nn_metrics)
  autolabel(line2, 0, lr_errs, lr_metrics)
  if use_knn: autolabel(line3, 1, knn_errs, knn_metrics)
  fig.tight_layout()

  ax.grid()
  plt.show()
  if save_folder != None:
    metric_ = metric.replace(" ", "_")
    name = f"{kind}_{metric_}_{fold2name(fold)}".lower()
    save_name = os.path.join(save_folder, name + "_model_line_comparison")
    fig.savefig(save_name+".png")
    with open(save_name + ".tex", "w") as file:
      text = """\\begin{figure}[H]
    \centering
    \includegraphics[width=0.97\\textwidth]{Images/%s.png}
    \caption{Comparação da métrica %s no conjunto de %s para %s}
    \label{fig:%s}
\end{figure}""" % (save_name.split("/")[-1], metric, fold2name(fold), kind.replace("_", "\_"), save_name.split("/")[-1])
      file.write(text)

  
for metric in metrics:
  plot_3_comparisons_line("INTER", metric=metric, save_folder=line_model_folder)
  plot_3_comparisons_line("INTRA", metric=metric, save_folder=line_model_folder)
  plot_3_comparisons_line("BACKWARD", metric=metric, use_knn=False, save_folder=line_model_folder)
  plot_3_comparisons_line("INTER_2", metric=metric, use_knn=False, save_folder=line_model_folder)
  plot_3_comparisons_line("INTER_500", metric=metric, save_folder=line_model_folder)

  

Output hidden; open in https://colab.research.google.com to view.

In [0]:
def write_txt(kind, fold="test", use_knn=True, metric="Balanced Accuracy", save_folder=None, plot_type="line"):  
  ctt_nn = (results.Experiment==f"NEURAL_{kind}") & (results.Fold==fold)
  ctt_knn = (results.Experiment==f"KNN_{kind}") & (results.Fold==fold)
  ctt_lr = (results.Experiment==f"LOGISTIC_{kind}") & (results.Fold==fold)

  use_knn = ctt_knn.sum()>0  
  
  tab_name_lr = results[ctt_lr].Experiment.unique()[0].lower() + "_results"
  if use_knn: tab_name_knn = results[ctt_knn].Experiment.unique()[0].lower() + "_results"
  tab_name_nn = results[ctt_nn].Experiment.unique()[0].lower() + "_results"
  
  metric_ = metric.replace(" ", "_")
  fig_name = f"{kind}_{metric_}_teste_model_{plot_type}_comparison".lower()
  
  labels = results[ctt_nn]["N"]
  nn = results[ctt_nn]
  knn = results[ctt_knn]
  lr= results[ctt_lr]

  nn_errs = nn[f"{metric} Std"]
  lr_errs = lr[f"{metric} Std"]
  if use_knn: knn_errs = knn[f"{metric} Std"]
  nn_metrics = nn[f"{metric} Mean"]
  lr_metrics = lr[f"{metric} Mean"]
  if use_knn: knn_metrics = knn[f"{metric} Mean"]
  
  def read_tex(filename, path):
    with open(os.path.join(path, filename + ".tex"), "r") as file:
      data = file.read()
    return "\n" + data
  
  path = f"/content/drive/My Drive/TCC/Images/{plot_type.capitalize()}_Model_Comparisons"

  assert tab_name_lr+".tex" in os.listdir(tables_path)
  assert tab_name_nn+".tex" in os.listdir(tables_path)
  if use_knn: assert tab_name_knn+".tex" in os.listdir(tables_path)
  assert fig_name+".tex" in os.listdir(path)

  kind_ = kind.replace("_","\_")
  
  if use_knn:
#     txt = """Os resultados a seguir referem-se ao experimento %s. Na tabela \\ref{tab:%s} estão os resultados para a regressão logística, enquanto a tabela \\ref{tab:%s} mostra os resultados para o k-NN, e a tabela \\ref{tab:%s} contém os resultados da rede neural. A figura \\ref{fig:%s} mostra a acurácia balanceada para cada modelo com cada número de movimentos sendo classificados.\n""" % (kind, tab_name_lr, tab_name_knn, tab_name_nn, fig_name)
    txt = """\section{%s}\n\label{section:%s}\nA tabela \\ref{tab:%s} mostra todas as métricas para o modelo de regressão logística, para todas as quantidades de classes, a tabela \\ref{tab:%s} mostra estes resultados obtidos com o modelo de \\textit{k-Nearest Neighbors}, e a tabela \\ref{tab:%s} contém os resultados da rede neural. A figura \\ref{fig:%s} mostra a acurácia balanceada para cada modelo com cada número de movimentos sendo classificados.\n""" % (kind_, kind, tab_name_lr, tab_name_knn, tab_name_nn, fig_name)
#     includes = ["\input{Images/%s.tex}" % fig_name,
#                 "\input{Images/%s.tex}" % tab_name_lr,
#                 "\input{Images/%s.tex}" % tab_name_knn,
#                 "\input{Images/%s.tex}" % tab_name_nn]
    includes = [read_tex(fig_name, path),
                read_tex(tab_name_lr, tables_path),
                read_tex(tab_name_nn, tables_path),
                read_tex(tab_name_knn, tables_path)]
    txt = txt + "\n".join(includes)
  else:
#     txt = """Os resultados a seguir referem-se ao experimento %s. Na tabela \\ref{tab:%s} estão os resultados para a regressão logística, a tabela \\ref{tab:%s} contém os resultados da rede neural e o algoritmo de k-NN não foi utilizado neste experimento. A figura \\ref{fig:%s} mostra a acurácia balanceada para cada modelo com cada número de movimentos sendo classificados.\n""" % (kind, tab_name_lr, tab_name_nn, fig_name)
    txt = """\section{%s}\n\label{section:%s}\nA tabela \\ref{tab:%s} mostra todas as métricas para o modelo de regressão logística, para todas as quantidades de classes, e a tabela \\ref{tab:%s} mostra estes resultados obtidos com o modelo de rede neural. A figura \\ref{fig:%s} mostra a acurácia balanceada para cada modelo com cada número de movimentos sendo classificados. Neste experimento não foi utilizado o modelo de \\textit{k-Nearest Neighbors}.\n""" % (kind_, kind, tab_name_lr, tab_name_nn, fig_name)
#     includes = ["\input{Images/%s.tex}" % fig_name,
#                 "\input{Images/%s.tex}" % tab_name_lr,
#                 "\input{Images/%s.tex}" % tab_name_nn]
    includes = [read_tex(fig_name, path),
                read_tex(tab_name_lr, tables_path),
                read_tex(tab_name_nn, tables_path)]
    txt = txt + "\n".join(includes)
  print(txt)
  
for kind in ["INTER", "INTER_500", "INTRA", "BACKWARD"]:
  print()
  write_txt(kind, plot_type="bar")


\section{INTER}
\label{section:INTER}
A tabela \ref{tab:logistic_inter_results} mostra todas as métricas para o modelo de regressão logística, para todas as quantidades de classes, a tabela \ref{tab:knn_inter_results} mostra estes resultados obtidos com o modelo de \textit{k-Nearest Neighbors}, e a tabela \ref{tab:neural_inter_results} contém os resultados da rede neural. A figura \ref{fig:inter_balanced_accuracy_teste_model_bar_comparison} mostra a acurácia balanceada para cada modelo com cada número de movimentos sendo classificados.

\begin{figure}[H]
    \centering
    \includegraphics[width=0.97\textwidth]{Images/inter_balanced_accuracy_teste_model_bar_comparison.png}
    \caption{Comparação da métrica \textit{balanced accuracy} no conjunto de teste para INTER}
    \label{fig:inter_balanced_accuracy_teste_model_bar_comparison}
\end{figure}

\begin{table}%
[H]%
\centering%
\caption{Resultados de LOGISTIC\_INTER}%
\label{tab:logistic_inter_results}%
\resizebox{\textwidth}{!}{%
\beg

In [0]:
def get_tex(folder):
  tex_files = [x for x in os.listdir(folder) if x.endswith(".tex")]
  text = []
  for x in tex_files:
    text.append("\include{Images/%s}" % x)
  return "\n".join(text)
get_tex(image_folder)
get_tex(comparison_folder)


''

In [0]:
mkdir "/content/drive/My Drive/TCC/Images/Results"

In [0]:
ls "/content/drive/My Drive/TCC/Images/Results"

NEURAL_INTER_logs.png
